In [3]:
import feedparser
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()

clean_path = "../data/clean/"
raw_path = "../data/raw/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
pdf_path = "../PDF/"

pd.set_option('display.max_colwidth', 255)
pd.set_option('display.max_rows',None)
url = "https://feeds.feedburner.com/Setorth-form45-en"

today = date.today()
year = 2021
mmdd_str = today.strftime('%m%d')
mmdd_str

'0214'

In [4]:
#today = date(2022, 2, 11)
mmdd_str = today.strftime('%m%d')
mmdd_str

'0214'

In [5]:
rss_source = feedparser.parse(url)
f45_number = len(rss_source.entries)
print("Number of F45: ", f45_number)

Number of F45:  32


In [6]:
f45_items = []

for x in range(f45_number):
    f45_content = rss_source.entries[x]
    f45_item = {}
    
    print("\n----------------------------------\n")
    
    print("F45: " + str(x))
    title_ary = f45_content.title.partition(' ')
    f45_item['name'] = title_ary[0].strip() 
    print("Title: ", f45_item['name'])  
    f45_item['year'] = year
    print("Year: ", f45_item['year'])      
    qtr_ary = title_ary[2].partition(' (F45)')
    f45_item['quarter'] = qtr_ary[0][-1]    
    print("Quarter: ", f45_item['quarter'])    
    f45_item['link'] = f45_content.link
    print("Link: ", f45_item['link'])
    f45_item['published'] = f45_content.published
    print("Published: ", f45_item['published'])  
    f45_items.append(f45_item)


----------------------------------

F45: 0
Title:  SMPC
Year:  2021
Quarter:  y
Link:  https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960136460
Published:  Mon, 14 Feb 2022 21:12:12 +0700

----------------------------------

F45: 1
Title:  SCGP
Year:  2021
Quarter:  S
Link:  https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960136200
Published:  Mon, 14 Feb 2022 21:08:13 +0700

----------------------------------

F45: 2
Title:  BTS
Year:  2021
Quarter:  3
Link:  https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960135510
Published:  Mon, 14 Feb 2022 20:47:21 +0700

----------------------------------

F45: 3
Title:  U
Year:  2021
Quarter:  y
Link:  https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960135360
Published:  Mon, 14 Feb 2022 20:35:03 +0700

----------------------------------

F45: 4
Title:  MTC
Year:  2021
Quarter:  y
Link:  https://www.set.or.th/set/newsdetails.do?lang

In [7]:
df = pd.DataFrame(f45_items)
df[['name','year','quarter','published']]

,name,year,quarter,published
0,SMPC,2021,y,"Mon, 14 Feb 2022 21:12:12 +0700"
1,SCGP,2021,S,"Mon, 14 Feb 2022 21:08:13 +0700"
2,BTS,2021,3,"Mon, 14 Feb 2022 20:47:21 +0700"
3,U,2021,y,"Mon, 14 Feb 2022 20:35:03 +0700"
4,MTC,2021,y,"Mon, 14 Feb 2022 19:51:24 +0700"
5,S11,2021,y,"Mon, 14 Feb 2022 19:47:52 +0700"
6,THIP,2021,y,"Mon, 14 Feb 2022 19:45:34 +0700"
7,TR,2021,3,"Mon, 14 Feb 2022 19:40:14 +0700"
8,PSL,2021,e,"Mon, 14 Feb 2022 19:34:06 +0700"
9,BLAND,2021,3,"Mon, 14 Feb 2022 18:51:51 +0700"


In [11]:
#df.iloc[44,2] = [3]
df.loc[df.quarter == ')','quarter'] = 4
#df.replace({"quarter" : "1"}, "3", inplace=True)

df

,name,year,quarter,link,published
0,SMPC,2021,4,https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960136460,"Mon, 14 Feb 2022 21:12:12 +0700"
1,SCGP,2021,4,https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960136200,"Mon, 14 Feb 2022 21:08:13 +0700"
2,BTS,2021,3,https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960135510,"Mon, 14 Feb 2022 20:47:21 +0700"
3,U,2021,4,https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960135360,"Mon, 14 Feb 2022 20:35:03 +0700"
4,MTC,2021,4,https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960134050,"Mon, 14 Feb 2022 19:51:24 +0700"
5,S11,2021,4,https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133820,"Mon, 14 Feb 2022 19:47:52 +0700"
6,THIP,2021,4,https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133660,"Mon, 14 Feb 2022 19:45:34 +0700"
7,TR,2021,3,https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133350,"Mon, 14 Feb 2022 19:40:14 +0700"
8,PSL,2021,4,https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133060,"Mon, 14 Feb 2022 19:34:06 +0700"
9,BLAND,2021,3,https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960130100,"Mon, 14 Feb 2022 18:51:51 +0700"


In [12]:
#df.iloc[44,2] = [3]
df.loc[df.quarter == '1','year'] = 2022
#df.replace({"quarter" : "1"}, "3", inplace=True)

df

,name,year,quarter,link,published
0,SMPC,2021,4,https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960136460,"Mon, 14 Feb 2022 21:12:12 +0700"
1,SCGP,2021,4,https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960136200,"Mon, 14 Feb 2022 21:08:13 +0700"
2,BTS,2021,3,https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960135510,"Mon, 14 Feb 2022 20:47:21 +0700"
3,U,2021,4,https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960135360,"Mon, 14 Feb 2022 20:35:03 +0700"
4,MTC,2021,4,https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960134050,"Mon, 14 Feb 2022 19:51:24 +0700"
5,S11,2021,4,https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133820,"Mon, 14 Feb 2022 19:47:52 +0700"
6,THIP,2021,4,https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133660,"Mon, 14 Feb 2022 19:45:34 +0700"
7,TR,2021,3,https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133350,"Mon, 14 Feb 2022 19:40:14 +0700"
8,PSL,2021,4,https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133060,"Mon, 14 Feb 2022 19:34:06 +0700"
9,BLAND,2021,3,https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960130100,"Mon, 14 Feb 2022 18:51:51 +0700"


In [13]:
df.shape

(32, 5)

In [14]:
### First equals to latest published pdf file
df = df.drop_duplicates(subset='name', keep='first')
df.shape

(28, 5)

In [15]:
file_name = 'F45-RAW-' + mmdd_str + '.csv'
raw_file = raw_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df[['name','year','quarter','published']].to_csv(output_file, header=True, index=False, sep=',')
df[['name','year','quarter','published']].to_csv(box_file,    header=True, index=False, sep=',')
df[['name','year','quarter','published']].to_csv(one_file,    header=True, index=False, sep=',')
df[['name','year','quarter','published']].to_csv(raw_file,    header=True, index=False, sep=',')

In [16]:
sql = '''
SELECT *
FROM exempts
ORDER BY name'''
df_exempts = pd.read_sql(sql, conlt)
df_exempts.shape[0]

299

In [17]:
df_merge = pd.merge(df, df_exempts, on='name', how='outer', indicator=True)
df_merge.shape

(324, 7)

### Tickers that won't be input

In [18]:
in_exempts = df_merge.loc[
    df_merge['_merge'] == 'both',
    ['name','year','quarter','published','link']
]
in_exempts.sort_values(by=['published'],ascending=[False])

,name,year,quarter,published,link
14,FUTUREPF,2021.0,4,"Mon, 14 Feb 2022 18:13:20 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960127470
24,TIF1,2021.0,3,"Mon, 14 Feb 2022 12:52:05 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960104740
25,QLT,2021.0,4,"Mon, 14 Feb 2022 08:55:03 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960096900


In [19]:
in_exempts.sort_values(by=['published'],ascending=[False]).shape[0]

3

In [20]:
df_out = df_merge.loc[
    df_merge['_merge'] == 'left_only',
    ['name','year','quarter','published','link']
]
df_out.sort_values(by=['published'],ascending=[False])

,name,year,quarter,published,link
0,SMPC,2021.0,4,"Mon, 14 Feb 2022 21:12:12 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960136460
1,SCGP,2021.0,4,"Mon, 14 Feb 2022 21:08:13 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960136200
2,BTS,2021.0,3,"Mon, 14 Feb 2022 20:47:21 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960135510
3,U,2021.0,4,"Mon, 14 Feb 2022 20:35:03 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960135360
4,MTC,2021.0,4,"Mon, 14 Feb 2022 19:51:24 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960134050
5,S11,2021.0,4,"Mon, 14 Feb 2022 19:47:52 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133820
6,THIP,2021.0,4,"Mon, 14 Feb 2022 19:45:34 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133660
7,TR,2021.0,3,"Mon, 14 Feb 2022 19:40:14 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133350
8,PSL,2021.0,4,"Mon, 14 Feb 2022 19:34:06 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133060
9,BLAND,2021.0,3,"Mon, 14 Feb 2022 18:51:51 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960130100


In [21]:
#df_out = df_out.drop(df_out.index[df_out['name'] == "SCC"])
df_out.shape[0]

25

In [22]:
sql = '''
SELECT *
FROM tickers
ORDER BY name'''
df_tickers = pd.read_sql(sql, conlt)
df_tickers.shape

(403, 9)

In [23]:
df_merge2 = pd.merge(df_out, df_tickers, on='name', how='outer', indicator=True)
df_merge2.shape

(408, 14)

### There are no ticker records

In [24]:
df_merge2.loc[
    df_merge2['_merge'] == 'left_only',
    ['name','year','quarter','published','link','id','market']
]

,name,year,quarter,published,link,id,market
6,THIP,2021.0,4,"Mon, 14 Feb 2022 19:45:34 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133660,NaN,NaN
18,TTT,2021.0,4,"Mon, 14 Feb 2022 17:12:25 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960120510,NaN,NaN
21,PROSPECT,2021.0,4,"Mon, 14 Feb 2022 13:06:47 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960105690,NaN,NaN
22,TLHPF,2021.0,4,"Mon, 14 Feb 2022 13:01:14 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960105370,NaN,NaN
23,PTC,2021.0,3,"Mon, 14 Feb 2022 08:24:12 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960095560,NaN,NaN


### There are ticker records

In [25]:
df_out2 = df_merge2.loc[
    df_merge2['_merge'] == 'both',
    ['name','year','quarter','published','link','id','market']
]
df_out2

,name,year,quarter,published,link,id,market
0,SMPC,2021.0,4,"Mon, 14 Feb 2022 21:12:12 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960136460,455.0,sSET
1,SCGP,2021.0,4,"Mon, 14 Feb 2022 21:08:13 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960136200,713.0,SET50 / SETCLMV / SETTHSI
2,BTS,2021.0,3,"Mon, 14 Feb 2022 20:47:21 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960135510,83.0,SET50 / SETTHSI / SETWB
3,U,2021.0,4,"Mon, 14 Feb 2022 20:35:03 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960135360,591.0,SET
4,MTC,2021.0,4,"Mon, 14 Feb 2022 19:51:24 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960134050,315.0,SET50 / SETTHSI
5,S11,2021.0,4,"Mon, 14 Feb 2022 19:47:52 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133820,412.0,sSET
7,TR,2021.0,3,"Mon, 14 Feb 2022 19:40:14 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133350,564.0,SET
8,PSL,2021.0,4,"Mon, 14 Feb 2022 19:34:06 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133060,734.0,SET
9,BLAND,2021.0,3,"Mon, 14 Feb 2022 18:51:51 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960130100,71.0,SET
10,COTTO,2022.0,1,"Mon, 14 Feb 2022 18:32:08 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960129040,710.0,SET


In [26]:
df_out2 = df_out2[df_out2.year.notnull()]
df_out2

,name,year,quarter,published,link,id,market
0,SMPC,2021.0,4,"Mon, 14 Feb 2022 21:12:12 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960136460,455.0,sSET
1,SCGP,2021.0,4,"Mon, 14 Feb 2022 21:08:13 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960136200,713.0,SET50 / SETCLMV / SETTHSI
2,BTS,2021.0,3,"Mon, 14 Feb 2022 20:47:21 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960135510,83.0,SET50 / SETTHSI / SETWB
3,U,2021.0,4,"Mon, 14 Feb 2022 20:35:03 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960135360,591.0,SET
4,MTC,2021.0,4,"Mon, 14 Feb 2022 19:51:24 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960134050,315.0,SET50 / SETTHSI
5,S11,2021.0,4,"Mon, 14 Feb 2022 19:47:52 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133820,412.0,sSET
7,TR,2021.0,3,"Mon, 14 Feb 2022 19:40:14 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133350,564.0,SET
8,PSL,2021.0,4,"Mon, 14 Feb 2022 19:34:06 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133060,734.0,SET
9,BLAND,2021.0,3,"Mon, 14 Feb 2022 18:51:51 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960130100,71.0,SET
10,COTTO,2022.0,1,"Mon, 14 Feb 2022 18:32:08 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960129040,710.0,SET


In [27]:
df_out2['year'] = df_out2['year'].astype(int)
df_out2['quarter'] = df_out2['quarter'].astype(int)
df_out2

,name,year,quarter,published,link,id,market
0,SMPC,2021,4,"Mon, 14 Feb 2022 21:12:12 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960136460,455.0,sSET
1,SCGP,2021,4,"Mon, 14 Feb 2022 21:08:13 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960136200,713.0,SET50 / SETCLMV / SETTHSI
2,BTS,2021,3,"Mon, 14 Feb 2022 20:47:21 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960135510,83.0,SET50 / SETTHSI / SETWB
3,U,2021,4,"Mon, 14 Feb 2022 20:35:03 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960135360,591.0,SET
4,MTC,2021,4,"Mon, 14 Feb 2022 19:51:24 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960134050,315.0,SET50 / SETTHSI
5,S11,2021,4,"Mon, 14 Feb 2022 19:47:52 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133820,412.0,sSET
7,TR,2021,3,"Mon, 14 Feb 2022 19:40:14 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133350,564.0,SET
8,PSL,2021,4,"Mon, 14 Feb 2022 19:34:06 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133060,734.0,SET
9,BLAND,2021,3,"Mon, 14 Feb 2022 18:51:51 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960130100,71.0,SET
10,COTTO,2022,1,"Mon, 14 Feb 2022 18:32:08 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960129040,710.0,SET


In [28]:
file_name = 'F45-CLEAN-' + mmdd_str + '.csv'
clean_file = clean_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(output_file, header=True, index=False, sep=',')
df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(clean_file, header=True, index=False, sep=',')
df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(box_file, header=True, index=False, sep=',')
df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(one_file, header=True, index=False, sep=',')

In [29]:
sql = '''
SELECT * 
FROM epss
WHERE year = 2021'''
df_epss = pd.read_sql(sql, conlt)
df_epss.shape

(1040, 14)

In [30]:
df_merge3 = pd.merge(df_out2, df_epss, on=['name','year','quarter'], how='outer', indicator=True)
df_merge3.shape

(1056, 19)

### Already input, display profit amt & eps to check with new F45

In [31]:
df_merge3[df_merge3['_merge'] == 'both']

,name,year,quarter,published,link,id_x,market,id_y,q_amt,y_amt,aq_amt,ay_amt,q_eps,y_eps,aq_eps,ay_eps,ticker_id,publish_date,_merge
1,SCGP,2021,4,"Mon, 14 Feb 2022 21:08:13 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960136200,713.0,SET50 / SETCLMV / SETTHSI,21092.0,2115508.0,1485991.0,8294367.0,6457475.0,0.4900,0.3600,1.9300,1.9500,713.0,2022-01-25,both
6,TR,2021,3,"Mon, 14 Feb 2022 19:40:14 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133350,564.0,SET,21022.0,1235872.0,183051.0,2474671.0,-468903.0,6.1300,0.9100,12.2800,-2.3300,564.0,2021-11-15,both
11,VGI,2021,3,"Mon, 14 Feb 2022 18:29:33 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960128770,609.0,SET100 / SETTHSI,20908.0,-9368.0,11858.0,680.0,-92094.0,-0.0001,0.0014,0.0001,-0.0107,609.0,2021-11-12,both
19,GLOBAL,2021,4,"Mon, 14 Feb 2022 07:23:47 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960093440,193.0,SET50 / SETCLMV / SETTHSI / SETWB,21116.0,746378.0,377592.0,3343575.0,1955963.0,0.1622,0.0821,0.7266,0.4251,193.0,2022-02-14,both


In [32]:
df_inp2 = df_merge3[df_merge3['_merge'] == 'left_only']
df_inp3 = df_inp2.copy()
df_inp3

,name,year,quarter,published,link,id_x,market,id_y,q_amt,y_amt,aq_amt,ay_amt,q_eps,y_eps,aq_eps,ay_eps,ticker_id,publish_date,_merge
0,SMPC,2021,4,"Mon, 14 Feb 2022 21:12:12 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960136460,455.0,sSET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,BTS,2021,3,"Mon, 14 Feb 2022 20:47:21 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960135510,83.0,SET50 / SETTHSI / SETWB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,U,2021,4,"Mon, 14 Feb 2022 20:35:03 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960135360,591.0,SET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,MTC,2021,4,"Mon, 14 Feb 2022 19:51:24 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960134050,315.0,SET50 / SETTHSI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
5,S11,2021,4,"Mon, 14 Feb 2022 19:47:52 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133820,412.0,sSET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
7,PSL,2021,4,"Mon, 14 Feb 2022 19:34:06 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133060,734.0,SET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
8,BLAND,2021,3,"Mon, 14 Feb 2022 18:51:51 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960130100,71.0,SET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
9,COTTO,2022,1,"Mon, 14 Feb 2022 18:32:08 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960129040,710.0,SET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
10,BCT,2021,3,"Mon, 14 Feb 2022 18:31:08 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960128900,729.0,SET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
12,MACO,2021,3,"Mon, 14 Feb 2022 18:25:17 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960128530,281.0,sSET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [33]:
df_inp3['year'] = df_inp3['year'].astype(str)
df_inp3['quarter'] = df_inp3['quarter'].astype(str)
inp3_str = df_inp3.name+' '+df_inp3.year+' '+df_inp3.quarter+' '+df_inp3.link
inp3_str

0       SMPC 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960136460
2        BTS 2021 3 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960135510
3          U 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960135360
4        MTC 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960134050
5        S11 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133820
7        PSL 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960133060
8      BLAND 2021 3 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960130100
9      COTTO 2022 1 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960129040
10       BCT 2021 3 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16447960128900
12      MACO 2021 3 https://www.set.or.th/set/